In [2]:
import laspy
import numpy as np
import shutil
import random

from laspy import LasData
from tqdm import tqdm
from pathlib import Path
from typing import List

In [3]:
AUGMENTED_PER_POINT_CLOUD = 15
MIN_TREES_PER_SCENE = 4
MAX_TREES_PER_SCENE = 15
MIN_DISTANCE = 5.0
VAL_CHUNKS = 32

YAW_RANGE = (0.0, 360.0)
TILT_RANGE = (-2.0, 2.0)
SCALE_RANGE = (0.9, 1.1)

mixed_dataset = Path.home() / 'Panoramix3D_data' / 'datasets' / 'EHydroDataset'

raw_folder = mixed_dataset / 'raw'
grounds_folder = raw_folder / 'grounds'
trees_folder = raw_folder / 'trees'

processed_folder = mixed_dataset / 'processed'
train_folder = processed_folder / 'train'
val_folder = processed_folder / 'val'
test_folder = processed_folder / 'test'

grounds_folder.mkdir(parents=True, exist_ok=True)
trees_folder.mkdir(parents=True, exist_ok=True)

processed_folder.mkdir(parents=True, exist_ok=True)
train_folder.mkdir(parents=True, exist_ok=True)
val_folder.mkdir(parents=True, exist_ok=True)
test_folder.mkdir(parents=True, exist_ok=True)

files = list((raw_folder / 'train').glob('*.las'))

ground_counter = 0
tree_counter = 0

In [4]:
def save_las(file, points, folder):
    out = laspy.create(point_format=file.point_format, file_version=file.header.version)
    out.header.scales = file.header.scales
    out.header.offsets = np.array([0.0, 0.0, 0.0])
    out.points = points

    x = points.x
    y = points.y
    z = points.z

    out.x = x - np.array(x).mean()
    out.y = y - np.array(y).mean()
    out.z = z - np.array(z).min()

    global ground_counter, tree_counter
    if folder == 'ground':
        out.write(grounds_folder / f'ground_{ground_counter}.las')
        ground_counter += 1
    else:
        out.write(trees_folder / f'tree_{tree_counter}.las')
        tree_counter += 1

def find_n_points(points, n):
    if len(points) <= n:
        return points
    
    choosen_idx = []
    idx = np.arange(len(points))
    np.random.shuffle(idx)
    idx = list(idx)

    while len(choosen_idx) < n and len(idx) > 0:
        candidate = idx.pop()
        too_close = False
        for c in choosen_idx:
            dist = np.linalg.norm(points[c] - points[candidate])
            if dist < MIN_DISTANCE:
                too_close = True
                break

        if not too_close:
            choosen_idx.append(candidate)

    return points[np.array(choosen_idx)]

def load_trees(paths: Path) -> List[LasData]:
    trees = []
    for path in paths:
        with laspy.open(path) as f:
            las_data = f.read()

        trees.append(las_data)
    return trees

def chunkerize_four(file):
    xy = np.stack([file.x, file.y], axis=1)
    center = xy.mean(axis=0)

    return [
        (xy[:, 0] > center[0]) & (xy[:, 1] > center[1]),
        (xy[:, 0] < center[0]) & (xy[:, 1] > center[1]),
        (xy[:, 0] < center[0]) & (xy[:, 1] < center[1]),
        (xy[:, 0] > center[0]) & (xy[:, 1] < center[1])
    ]


In [28]:
for file in tqdm(files):
    with laspy.open(file) as f:
        ground = f.read()

    ground_mask = ground.semantic_gt == 0
    ground_points = ground.points[ground_mask]
    save_las(ground, ground_points, 'ground')

    unique_ids = np.unique(ground.instance_gt)
    unique_ids = unique_ids[unique_ids > 0]

    for id in unique_ids:
        tree_mask = ground.instance_gt == id
        tree_xyz = ground.points[tree_mask]
        save_las(ground, tree_xyz, 'tree')


100%|██████████| 24/24 [00:30<00:00,  1.26s/it]


In [15]:
ground_files = list((grounds_folder).glob('*.las'))
tree_files = list((trees_folder).glob('*.las'))

for file in tqdm(range(len(ground_files))):
    # print(ground_files[file])
    with laspy.open(ground_files[file]) as f:
        ground = f.read()

    ground_scale = np.random.uniform(*SCALE_RANGE)
    ground_yaw = np.radians(np.random.uniform(*YAW_RANGE))
    ground_tilt = np.radians(np.random.uniform(*TILT_RANGE))

    xyz = np.array(ground.xyz) * ground_scale
    xyz = xyz @ np.array([
        [np.cos(ground_yaw), -np.sin(ground_yaw), 0],
        [np.sin(ground_yaw), np.cos(ground_yaw), 0],
        [0, 0, 1]
    ])
    xyz = xyz @ np.array([
        [1, 0, 0],
        [0, np.cos(ground_tilt), -np.sin(ground_tilt)],
        [0, np.sin(ground_tilt), np.cos(ground_tilt)]
    ])

    ground.xyz = xyz

    for copy in range(AUGMENTED_PER_POINT_CLOUD):
        num_trees = np.random.randint(MIN_TREES_PER_SCENE, MAX_TREES_PER_SCENE + 1)
        choosen_points = find_n_points(xyz[ground.semantic_gt == 0], num_trees)
        choosen_trees = np.random.choice(tree_files, num_trees, replace=False)
        trees = load_trees(choosen_trees)

        out_path = raw_folder / 'aug' / f'{ground_files[file].stem}_aug_{copy}.las'
        with laspy.open(out_path, mode='w', header=ground.header) as w:
            w.write_points(ground.points)

            for id, (point, tree) in enumerate(zip(choosen_points, trees)):
                tree.instance_gt = np.full_like(tree.instance_gt, fill_value=id + 1)
                tree.semantic_gt = np.full_like(tree.semantic_gt, fill_value=5)

                tree_scale = np.random.uniform(*SCALE_RANGE)
                tree_yaw = np.radians(np.random.uniform(*YAW_RANGE))
                tree_tilt = np.radians(np.random.uniform(*TILT_RANGE))
                
                tree_xyz = np.array(tree.xyz) * tree_scale
                tree_xyz = tree_xyz @ np.array([
                    [np.cos(tree_yaw), -np.sin(tree_yaw), 0],
                    [np.sin(tree_yaw), np.cos(tree_yaw), 0],
                    [0, 0, 1]
                ])
                tree_xyz = tree_xyz @ np.array([
                    [1, 0, 0],
                    [0, np.cos(tree_tilt), -np.sin(tree_tilt)],
                    [0, np.sin(tree_tilt), np.cos(tree_tilt)]
                ])

                tree_xyz += point
                tree.xyz = tree_xyz

                w.write_points(tree.points)
    

100%|██████████| 64/64 [00:17<00:00,  3.69it/s]


In [16]:
folders = ['train', 'val', 'test', 'aug']
for folder in tqdm(folders):
    split_folder = raw_folder / folder
    files = list((split_folder).glob('*.las'))

    for file in files:
        with laspy.open(file) as f:
            las = f.read()

        masks = chunkerize_four(las)
        for i, mask in enumerate(masks):
            chunk = las.points[mask]
            out = laspy.create(point_format=las.point_format, file_version=las.header.version)
            out.header.scales = las.header.scales
            out.header.offsets = np.array([0.0, 0.0, 0.0])
            out.points = chunk

            x = chunk.x
            y = chunk.y
            z = chunk.z

            out.x = x - np.array(x).min()
            out.y = y - np.array(y).min()
            out.z = z - np.array(z).min()

            _, out.instance_gt = np.unique(chunk.instance_gt, return_inverse=True)

            out_folder = (processed_folder / folder) if folder != 'aug' else (processed_folder / 'train')
            out.write(out_folder / f'{file.stem}_chunk_{i}.las')

100%|██████████| 4/4 [00:15<00:00,  3.90s/it]


In [5]:
train_files = list((processed_folder / 'train').glob('*.las'))
print(f"📁 Archivos encontrados en train: {len(train_files)}")

# Analizar qué clases contiene cada archivo
print("🔍 Analizando clases semánticas en cada archivo...")
file_classes = {}
all_classes = set(range(9))  # Clases [0, 1, 2, 3, 4, 5, 6, 7, 8]

for train_file in tqdm(train_files, desc="Analizando clases"):
    with laspy.open(train_file) as f:
        las = f.read()
    unique_classes = set(np.unique(las.semantic_gt))
    file_classes[train_file] = unique_classes

# Selección estratégica para asegurar que val tenga todas las clases
selected_files = []
val_classes_covered = set()

# Primero, seleccionar archivos que aporten clases faltantes
print("📊 Seleccionando archivos para cubrir todas las clases...")
for train_file, classes in sorted(file_classes.items(), key=lambda x: len(x[1]), reverse=True):
    if len(selected_files) >= VAL_CHUNKS:
        break
    
    # Si este archivo aporta clases nuevas o aún no tenemos todas las clases
    new_classes = classes - val_classes_covered
    if new_classes or val_classes_covered != all_classes:
        selected_files.append(train_file)
        val_classes_covered.update(classes)
        if new_classes:
            print(f"  ✅ {train_file.name} aporta clases: {sorted(new_classes)}")

# Si aún no tenemos suficientes archivos, añadir aleatoriamente
remaining_files = [f for f in train_files if f not in selected_files]
if len(selected_files) < VAL_CHUNKS and remaining_files:
    additional_needed = VAL_CHUNKS - len(selected_files)
    additional_files = random.sample(remaining_files, min(additional_needed, len(remaining_files)))
    selected_files.extend(additional_files)

print(f"\n🎯 Archivos seleccionados para mover a val: {len(selected_files)}")
print(f"📊 Clases cubiertas en val: {sorted(val_classes_covered)}")
missing_classes = all_classes - val_classes_covered
if missing_classes:
    print(f"⚠️  Clases NO cubiertas en val: {sorted(missing_classes)}")
else:
    print(f"✅ Todas las clases [0-8] están representadas en val")

# Mover los archivos seleccionados
moved_count = 0
for file_path in tqdm(selected_files, desc="Moviendo archivos a val"):
    try:
        destination = processed_folder / 'val' / file_path.name
        shutil.move(str(file_path), str(destination))
        moved_count += 1
    except Exception as e:
        print(f"❌ Error moviendo {file_path.name}: {e}")

print(f"\n✅ Se movieron {moved_count} archivos de train a val")
print(f"📊 Archivos restantes en train: {len(list((processed_folder / 'train').glob('*.las')))}")
print(f"📊 Archivos totales en val: {len(list((processed_folder / 'val').glob('*.las')))}")

📁 Archivos encontrados en train: 3840
🔍 Analizando clases semánticas en cada archivo...


Analizando clases:   0%|          | 0/3840 [00:00<?, ?it/s]

Analizando clases: 100%|██████████| 3840/3840 [00:01<00:00, 2063.99it/s]



📊 Seleccionando archivos para cubrir todas las clases...
  ✅ Grounds_seg_10_2025-11-17_19h51_34_773_aug_3_chunk_2.las aporta clases: [0, 2, 4, 5, 6, 7]
  ✅ Grounds_seg_5_2025-11-17_19h52_28_683_aug_12_chunk_0.las aporta clases: [8]
  ✅ Grounds_seg_3_2025-11-17_19h53_02_211_aug_6_chunk_1.las aporta clases: [1, 3]

🎯 Archivos seleccionados para mover a val: 32
📊 Clases cubiertas en val: [0, 1, 2, 3, 4, 5, 6, 7, 8]
✅ Todas las clases [0-8] están representadas en val


Moviendo archivos a val: 100%|██████████| 32/32 [00:00<00:00, 7250.70it/s]


✅ Se movieron 32 archivos de train a val
📊 Archivos restantes en train: 3808
📊 Archivos totales en val: 32


In [ ]:
# Verificar IDs únicos en todas las nubes de aug
aug_folder = raw_folder / 'aug'
aug_files = list(aug_folder.glob('*.las'))

print(f"Analizando {len(aug_files)} archivos en la carpeta 'aug'...")

all_unique_ids = set()
file_id_info = []

for aug_file in tqdm(aug_files, desc="Checking aug files"):
    with laspy.open(aug_file) as f:
        las = f.read()
    
    unique_ids = np.unique(las.semantic_gt)
    all_unique_ids.update(unique_ids)
    
    file_id_info.append({
        'file': aug_file.name,
        'unique_ids': unique_ids,
        'min_id': np.min(unique_ids),
        'max_id': np.max(unique_ids),
        'num_unique': len(unique_ids)
    })

print(f"\n📊 RESUMEN GLOBAL:")
print(f"IDs únicos encontrados en todo el conjunto 'aug': {sorted(all_unique_ids)}")
print(f"Rango de IDs: {min(all_unique_ids)} - {max(all_unique_ids)}")
print(f"Total de IDs únicos diferentes: {len(all_unique_ids)}")

print(f"\n📋 DETALLES POR ARCHIVO (primeros 10):")
for i, info in enumerate(file_id_info[:10]):
    print(f"{info['file']}: IDs {info['unique_ids']} (rango: {info['min_id']}-{info['max_id']}, total: {info['num_unique']})")

Analizando 960 archivos en la carpeta 'aug'...


Checking aug files: 100%|██████████| 960/960 [00:01<00:00, 855.61it/s]


📊 RESUMEN GLOBAL:
IDs únicos encontrados en todo el conjunto 'aug': [0, 1, 2]
Rango de IDs: 0 - 2
Total de IDs únicos diferentes: 3

📋 DETALLES POR ARCHIVO (primeros 10):
Grounds_seg_0_2025-11-17_19h52_28_526_aug_7.las: IDs [0 1 2] (rango: 0-2, total: 3)
Grounds_seg_5_2025-11-17_19h53_02_269_aug_3.las: IDs [0 1 2] (rango: 0-2, total: 3)
Grounds_seg_10_2025-11-17_19h52_28_825_aug_13.las: IDs [0 1 2] (rango: 0-2, total: 3)
Grounds_seg_6_2025-11-17_19h53_19_952_aug_12.las: IDs [0 1 2] (rango: 0-2, total: 3)
Grounds_seg_3_2025-11-17_19h53_26_029_aug_0.las: IDs [0 1 2] (rango: 0-2, total: 3)
Grounds_seg_0_2025-11-17_19h53_19_793_aug_2.las: IDs [0 1 2] (rango: 0-2, total: 3)
Grounds_seg_6_2025-11-17_19h53_26_102_aug_7.las: IDs [0 1 2] (rango: 0-2, total: 3)
Grounds_seg_9_2025-11-17_19h52_28_801_aug_11.las: IDs [0 1 2] (rango: 0-2, total: 3)
Grounds_seg_3_2025-11-17_19h52_07_046_aug_14.las: IDs [0 1 2] (rango: 0-2, total: 3)
Grounds_seg_1_2025-11-17_19h51_54_601_aug_5.las: IDs [0 1 2] (rango

In [ ]:
# Verificar consistencia semantic_gt vs instance_gt en train
train_files = list((processed_folder / 'train').glob('*.las'))
print(f"Verificando {len(train_files)} archivos en train...")

problematic_files = []

for train_file in tqdm(train_files, desc="Verificando consistencia semantic/instance"):
    with laspy.open(train_file) as f:
        las = f.read()
    
    semantic_gt = np.array(las.semantic_gt)
    instance_gt = np.array(las.instance_gt)
    
    # Regla 1: semantic_gt == 5 debe tener instance_gt != 0
    mask_semantic_5 = semantic_gt == 5
    mask_instance_0_but_semantic_5 = mask_semantic_5 & (instance_gt <= 0)
    num_violations_rule1 = np.sum(mask_instance_0_but_semantic_5)
    
    # Regla 2: semantic_gt != 5 debe tener instance_gt == 0
    mask_semantic_not_5 = semantic_gt != 5
    mask_instance_not_0_but_semantic_not_5 = mask_semantic_not_5 & (instance_gt != 0)
    num_violations_rule2 = np.sum(mask_instance_not_0_but_semantic_not_5)
    
    if num_violations_rule1 > 0 or num_violations_rule2 > 0:
        problematic_files.append({
            'file': train_file.name,
            'rule1_violations': num_violations_rule1,  # semantic=5 pero instance=0
            'rule2_violations': num_violations_rule2,  # semantic!=5 pero instance!=0
            'total_points': len(semantic_gt)
        })

print(f"\n{'='*80}")
print(f"📊 RESUMEN:")
print(f"Total de archivos verificados: {len(train_files)}")
print(f"Archivos con problemas: {len(problematic_files)}")
print(f"Archivos correctos: {len(train_files) - len(problematic_files)}")

if problematic_files:
    print(f"\n❌ ARCHIVOS PROBLEMÁTICOS (primeros 20):")
    print(f"{'='*80}")
    for i, info in enumerate(problematic_files[:20]):
        print(f"\n{info['file']}:")
        if info['rule1_violations'] > 0:
            pct1 = (info['rule1_violations'] / info['total_points']) * 100
            print(f"  ❌ Regla 1: {info['rule1_violations']} puntos con semantic=5 pero instance=0 ({pct1:.2f}%)")
        if info['rule2_violations'] > 0:
            pct2 = (info['rule2_violations'] / info['total_points']) * 100
            print(f"  ❌ Regla 2: {info['rule2_violations']} puntos con semantic!=5 pero instance!=0 ({pct2:.2f}%)")
    
    if len(problematic_files) > 20:
        print(f"\n... y {len(problematic_files) - 20} archivos más con problemas")
else:
    print(f"\n✅ Todos los archivos cumplen las reglas:")
    print(f"   - semantic_gt == 5 → instance_gt != 0")
    print(f"   - semantic_gt != 5 → instance_gt <= 0")

Verificando 3808 archivos en train...


Verificando consistencia semantic/instance:   0%|          | 0/3808 [00:00<?, ?it/s]

Verificando consistencia semantic/instance: 100%|██████████| 3808/3808 [00:01<00:00, 2385.53it/s]


📊 RESUMEN:
Total de archivos verificados: 3808
Archivos con problemas: 0
Archivos correctos: 3808

✅ Todos los archivos cumplen las reglas:
   - semantic_gt == 5 → instance_gt != 0
   - semantic_gt != 5 → instance_gt == 0


In [22]:
# Verificar consistencia semantic_gt vs species_gt en train
train_files = list((processed_folder / 'train').glob('*.las'))
print(f"Verificando {len(train_files)} archivos en train...")

problematic_files_species = []

for train_file in tqdm(train_files, desc="Verificando consistencia semantic/species"):
    with laspy.open(train_file) as f:
        las = f.read()
    
    semantic_gt = np.array(las.semantic_gt)
    species_gt = np.array(las.species_gt)
    
    # Regla 1: semantic_gt == 5 debe tener species_gt > 0
    mask_semantic_5 = semantic_gt == 5
    mask_species_0_but_semantic_5 = mask_semantic_5 & (species_gt == 0)
    num_violations_rule1 = np.sum(mask_species_0_but_semantic_5)
    
    # Regla 2: semantic_gt != 5 debe tener species_gt == 0
    mask_semantic_not_5 = semantic_gt != 5
    mask_species_not_0_but_semantic_not_5 = mask_semantic_not_5 & (species_gt != 0)
    num_violations_rule2 = np.sum(mask_species_not_0_but_semantic_not_5)
    
    if num_violations_rule1 > 0 or num_violations_rule2 > 0:
        problematic_files_species.append({
            'file': train_file.name,
            'rule1_violations': num_violations_rule1,  # semantic=5 pero species=0
            'rule2_violations': num_violations_rule2,  # semantic!=5 pero species!=0
            'total_points': len(semantic_gt)
        })

print(f"\n{'='*80}")
print(f"📊 RESUMEN:")
print(f"Total de archivos verificados: {len(train_files)}")
print(f"Archivos con problemas: {len(problematic_files_species)}")
print(f"Archivos correctos: {len(train_files) - len(problematic_files_species)}")

if problematic_files_species:
    print(f"\n❌ ARCHIVOS PROBLEMÁTICOS (primeros 20):")
    print(f"{'='*80}")
    for i, info in enumerate(problematic_files_species[:20]):
        print(f"\n{info['file']}:")
        if info['rule1_violations'] > 0:
            pct1 = (info['rule1_violations'] / info['total_points']) * 100
            print(f"  ❌ Regla 1: {info['rule1_violations']} puntos con semantic=5 pero species=0 ({pct1:.2f}%)")
        if info['rule2_violations'] > 0:
            pct2 = (info['rule2_violations'] / info['total_points']) * 100
            print(f"  ❌ Regla 2: {info['rule2_violations']} puntos con semantic!=5 pero species!=0 ({pct2:.2f}%)")
    
    if len(problematic_files_species) > 20:
        print(f"\n... y {len(problematic_files_species) - 20} archivos más con problemas")
else:
    print(f"\n✅ Todos los archivos cumplen las reglas:")
    print(f"   - semantic_gt == 5 → species_gt > 0")
    print(f"   - semantic_gt != 5 → species_gt == 0")

Verificando 3808 archivos en train...


Verificando consistencia semantic/species: 100%|██████████| 3808/3808 [00:01<00:00, 2755.66it/s]


📊 RESUMEN:
Total de archivos verificados: 3808
Archivos con problemas: 0
Archivos correctos: 3808

✅ Todos los archivos cumplen las reglas:
   - semantic_gt == 5 → species_gt > 0
   - semantic_gt != 5 → species_gt == 0


In [ ]:
# Calcular desbalance de clases en train
train_files = list((processed_folder / 'train').glob('*.las'))
print(f"📊 Calculando desbalance de clases en {len(train_files)} archivos de train...")

# Contador de puntos por clase
class_counts = {i: 0 for i in range(9)}  # Clases [0-8]
total_points = 0

for train_file in tqdm(train_files, desc="Contando puntos por clase"):
    with laspy.open(train_file) as f:
        las = f.read()
    
    semantic_gt = np.array(las.semantic_gt)
    unique, counts = np.unique(semantic_gt, return_counts=True)
    
    for cls, count in zip(unique, counts):
        if cls in class_counts:
            class_counts[cls] += count
    
    total_points += len(semantic_gt)

print(f"\n{'='*80}")
print(f"📈 DISTRIBUCIÓN DE CLASES EN TRAIN:")
print(f"{'='*80}")
print(f"Total de puntos: {total_points:,}")
print()

# Ordenar por número de clase
for cls in sorted(class_counts.keys()):
    count = class_counts[cls]
    percentage = (count / total_points) * 100 if total_points > 0 else 0
    bar = '█' * int(percentage / 2)  # Barra visual (cada █ = 2%)
    print(f"Clase {cls}: {count:>12,} puntos ({percentage:>6.2f}%) {bar}")

print(f"\n{'='*80}")
print(f"📊 ANÁLISIS DE DESBALANCE:")
print(f"{'='*80}")

# Clase más frecuente y menos frecuente
max_class = max(class_counts, key=class_counts.get)
min_class = min(class_counts, key=class_counts.get)
max_count = class_counts[max_class]
min_count = class_counts[min_class]

print(f"Clase más frecuente: {max_class} ({max_count:,} puntos)")
print(f"Clase menos frecuente: {min_class} ({min_count:,} puntos)")

if min_count > 0:
    imbalance_ratio = max_count / min_count
    print(f"Ratio de desbalance: {imbalance_ratio:.2f}:1")
else:
    print(f"⚠️  Clase {min_class} no tiene puntos!")

# Calcular pesos para balanceo (inversamente proporcional a la frecuencia)
print(f"\n💡 PESOS SUGERIDOS PARA BALANCEO (inversamente proporcional):")
weights = {}
for cls in sorted(class_counts.keys()):
    if class_counts[cls] > 0:
        weight = total_points / (len(class_counts) * class_counts[cls])
        weights[cls] = weight
        print(f"Clase {cls}: {weight:.4f}")
    else:
        print(f"Clase {cls}: N/A (sin puntos)")